# Overview

This is a showcase how to control agent's output of your LangChain custom tools in cases where their output may not be very specific, but you still want to get it in a very specific format.

For example, you want the tool to access your content database, do something with its entries and have the agent output the processed content in, say, Markdown format.


## Loading libraries and variables

In [21]:
# Loading libraries

from dotenv import load_dotenv
from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import BaseTool

In [22]:
# Loading variables from .env (like OPENAI_API_KEY and other ones that are required)

load_dotenv()

True


## Creating LangChain custom tool

Read the comment to this part of the code to learn how to have your agent answer in a predictable format.

In [27]:
class ContentOverviewTool(BaseTool):
    name = "Content overviewer"
    description = "use this tool when you need to overview a piece of content from the database, identified by source_id"


    # In this example I'm not using source_id, but in the real-case scenario
    # this would be an identifier to find content in the database
    def _run(self, source_id=None):

        # Here for the sake of the example I'm hard-coding the content (in this case - a transcript of one of David Shapiro's video's)
        # into the variable, but in my real apps I query database (relational or vector) to get the required content piece.
        content = """Good morning everybody, David Shapiro here. So in the last 24 hours, we have seen several interesting pieces of news drop. OpenAI, Tesla, and IBM are all throwing around numbers in the millions and billions, and this is in terms of valuation, revenue, and investment. So let's get right into it. First up, IBM has invested even more into Hugging Face. So they have been involved with Hugging Face. For instance, they're collaborating on Watson X. However, they just closed a Series D funding round for $235 million, valuing Hugging Face at $4.5 billion. So this is a startup that produces primarily open-source AI and has only been around for, what, two years, I think? Maybe three, at least that's as long as I've kind of known about them, as long as they've been on my radar. Now, at the same time, the CEO of IBM, Arvind Krishna, has frozen thousands and thousands of jobs. There's 3,900 more layoffs planned at IBM, with 7,800 positions frozen. And this is also someone who has said that he expects 30 to 50% of repetitive tasks to be taken by AI, and that also, that those back-office tasks will probably be done better by AI. So better, faster, cheaper. In other words, the takeaway, IBM is doubling down on AI. Next up is Tesla. So Tesla just fired up their high-performance computing cluster. It's worth $300 million, and it is built of 10,000 NVIDIA H100 GPUs. So this is primarily gonna be used to take on general-purpose HPC workloads. So this is an HPC compute cluster, which basically means that it is high-performance Tensor cores that you can use for anything. Now, obviously, Tesla, they do a lot of AI, but their flagship AI product is FSD, which is full self-driving. Elon also said that they are going to invest $4 billion more into AI over the next two years, including at least a billion dollars into their Dojo supercomputer, which is their primary training computer. So again, we see tech giants doubling down on AI with numbers in the hundreds of millions and billions of dollars. And kind of the coup de grace of yesterday's news is that OpenAI's revenue has exploded. So last year, they made $28 million in revenue, and in the next 12 months, they're on track to make a billion dollars in revenue. Their revenue has obviously exploded, primarily due to chat GPT and having more than 100 million users sign up in two months. But one thing that you need to keep in mind is that this is a 35x increase in revenue growth. So this is where tech companies really aim for to be like a true unicorn. So the technical definition of unicorn is any tech startup that gets to a billion-dollar valuation. But what the next phase that you're looking for is for them to take off exponentially. This is the nature of the book Exponential Organizations, which I bought, but I haven't fully read yet. Anyways, the idea is that because of the nature of technology, because user adoption can be that fast, because you can immediately have global exposure once your product hits the market, that's how you can have 35x revenue growth in tech and in literally no other industry. Because in most other industries where there is human labor involved and when there is physical goods and locations involved, it's just not possible to grow that fast. One part of the reports out there says that ChatGPT costs them $700,000 per day to run. And if you do the math, I did some quick back-of-the-napkin calculations, if ChatGPT costs $700,000 a day to run, then you add it all up and you look at a billion-dollar revenue, it probably costs ChatGPT at current rates about $250 to $300 million per year to run. So if their revenue is one billion, but the outflow's just from running ChatGPT, which again, right now is their flagship product, then that means they still have $700 million left over for things like employee revenue or employee salaries, research, that sort of stuff. Now, another thing to make this more complicated is that because of the deal that OpenAI signed with Microsoft, obviously I haven't read the entire full package of their deal, but my understanding is that Microsoft is entitled to up to 75% of OpenAI's revenue until they reach a threshold, and I think that that threshold is until OpenAI earns them $100 billion. So originally, as of last year, they thought that it could take a decade or more to pay it off. Now, given this recent news and the explosion, maybe that timeframe will shorten. Now, obviously, a billion dollars in revenue, if they're giving less than that to Microsoft, that's still gonna be like 100 years. So if their revenue increases to 10 billion a year or something like that, then they'll be able to pay it off even faster. Again, I'm not sure of all the full terms, but my point here is that Microsoft is benefiting from their investment. Now, I don't know if the whole thing is cash positive. OpenAI might still be losing money. They certainly were last year. Microsoft might still be losing money on the deal, but what they're doing is these are strategic moves that the companies are doing in order to better position themselves as these technologies ramp up. Okay, so that's the three pieces of news. Let's dive into the analysis. First is data and metrics and predictions on investment growth. So when you see numbers like this, you know, 235 million, 300 million, a billion dollars, that is really interesting because this is kind of, I'm not gonna say it's the first time we're seeing numbers this big, but we're certainly seeing numbers this big in much shorter periods of time. And so this made me like look into like, okay, what did the financial analysts predict? And it looks like we're actually kind of on track. So yes, these are like exciting numbers, but Goldman Sachs forecasted last year that by 2025, we should be seeing $200 billion per year in these kinds of AI investments, which is double what it was, more than double what it was last year of 92 billion. There is some evidence that 2023 might be seeing a slowdown, but there's also some evidence that it might be accelerating. So we're not really gonna know until the year is over and we've had a chance to tally it all up. But right now, the general consensus seems to be that things are chugging along more or less as expected. So keep these numbers in mind that like, yes, the growth is explosive here. So it's a really good time to be selling AI goods, services, hardware, software, that sort of stuff, because the investment is coming, the investment is growing. And so all these moves that we're seeing are just naturally to be expected, because if we go from 66 billion to 92 billion one year, and then we go from 92 billion to let's say 120 billion this year, that's 30 billion more dollars that's gonna be thrown around on top of what was thrown around last year. So the investment is happening. We're seeing this exponential ramp up. I guess this is more of a geometric ramp up rather, adding about $30 billion per year of investment to AI resources. So we're seeing this geometric ramp up, but we're not talking in the millions of dollars, we're talking in the billions and very soon hundreds of billions of dollars, which means that any problems that AI has are going to be solved. And this goes back to my video a few weeks ago, AI winter is not coming. This is more evidence that AI winter is not coming. So do not think about AI as it is today. Think about AI as it will be a year from now. Because remember, this time last year, none of you knew, well, I'm not gonna say none of you, some of my longtime fans will remember when it was just GPT-3, because that's all we had this time last year. But then, come November, chat GPT came out and everything changed. So imagine where we're gonna be this time next year. Now, on top of investment, I'm one of the ones that's always beating the drum on post-labor economics and utopian future where nobody has to work. So I decided to put my money, or numbers rather, where my mouth is, and I looked it up. So I used perplexity to find all kinds of statistics from various dimensions and angles. And so yes, there have been 150,000 or more US tech layoffs as of June. So that's excluding July and now August. So there's two more months. However, the recent data shows that total unemployment in America remains at a relatively low 3.5%. There were 375,000 tech jobs open as of January this year. Obviously, that's probably changed a little bit. Forecasts at the same time showed that they were expecting 272,000 more new tech jobs by the end of 2023. So that remains to be seen. Those last two sources were not government sources. Those were from ZipRecruiter and stuff like that. So take those with a grain of salt. But the layoffs, that was government data. So that's pretty reliable data. Some other non-governmental sources said that AI is expected to do better.to destroy 85 million jobs globally by 2025 but is expected to create up to 97 million jobs globally by 2025. So that would be a net gain of 12 million jobs. And so when you see the layoffs and new jobs that are going unfulfilled, there's a trend emerging here which is that old jobs are going away and new jobs are being created. Again, many of these sources are not government. A lot of them are speculation and forecasting, so take it with a grain of salt. But the pattern to me seems relatively clear, and that is that we are going through a workforce transformation. So it is time to skill up. Another piece of evidence in this puzzle is that generative AI job postings jumped 20% in May alone, or maybe that was year over year. I don't remember. Anyways, you know, so post-labor economics not here yet, but there is evidence that you need to be pivoting to generative AI skills and tools. So if you're not, you're going to get further and further behind. Now, this slide was actually the most interesting thing to me because, you know, I talked to some of my fellow communicators and YouTubers and researchers. I talked to people all over the world. And so if you talk to individuals and figure out what their concerns are, you know, like government policymakers trying to get, you know, other people in the government on board with AI, or business leaders trying to figure out how to adopt AI, the numbers of public sentiment were actually more where I was hoping they would be, which kind of invalidated some of the work that I've done. But hey, this is a good place for me to be wrong. So let me dive into the data. A Reuters Ipsos poll recently showed that 61% of Americans view AI as a potential threat to human civilization. That is a huge, that is almost a super majority already of Americans that say, yes, AI could wipe us all out. So thanks to James Cameron and The Matrix and all that other, you know, every story out there, we have a schema, those fictional stories have done their job to prepare us for the potential dangers of AI. The next piece of information is that a Pew research poll showed that 58% of Americans are more concerned than excited about the rise of AI in daily life. So again, there's a little bit of anxiety and consternation rather than just raw excitement. So I'm not saying that like the doomers are winning. I'm just saying that like there's people are cautious. And then this one was far and away the most interesting, which was an economist and YouGov poll showed that about 75% of Americans believe that AI should be regulated or heavily regulated by the government. That's 79% Democrat and 73% Republicans. We rarely ever see this much consensus on anything in America. But when you have either a majority, near super majority or above super majority of Americans agreeing on something, the political willpower is there. So this gives me a tremendous amount of confidence that regulations are going to happen, that people are taking it seriously, and that this will help steer the ship in the right direction. So I wanted to share this information with you because to me, it gave me a lot of confidence that we're heading in the right direction. All right, so some conclusions and takeaways. Here's my predictions. AI investment may cool slightly, and I don't mean that it's going to go down. I just mean it's going to slow down. So it's not going to go negative. It's just it's not going to be quite as hot as it has been. And my interpretation is that the several class action lawsuits filed against OpenAI have kind of spooked the markets and maybe made people a little bit more tepid about jumping in. Now, that being said, we just did see, you know, a lot of investment yesterday. So we'll see. Meta has been taking an entirely different approach to AI where they're just doing everything open source. It's a bold strategy. We'll see how it plays out. So stay tuned on the space of closed source versus open source. Personally, I think that open source is the way just because it's going to have less friction and it's going to have more collaboration, which means that any AI built on open source data and open source models are probably just going to be better, not the least of which they're going to be more democratic. That's my hope. Let's see. Number two, Americans are largely united on regulations and fears, but regulatory capture is still a primary concern. With regulation comes risk, because if you regulate it incorrectly, if you listen to the wrong people when you're coming up with regulations, then, you know, every stakeholder is going to try and stack the regulations in their favor. So stacking the deck in their favor, whether it's big companies, whether it's governments or whatever other special interests or lobbyists are out there, that's the primary concern. Now, the amount of attention that Americans are putting on AI I think means that we're going to demand transparency and we're going to demand accountability. So my hope is that this is something that we're going to do right. And probably what I'll do is because there's so much consensus on this, I'll do a follow-up video on how do we do AI regulation correctly so that we don't end up with regulatory capture. And finally, the state of jobs right now seems really good. So, you know, post-labor economics and UBI are going to have to wait for a while. Like I said in a previous video where I introduced these concepts, there's a few signals that we can pay attention to, such as deflation, jobs loss, and a few other things. We're still in a state of inflation and we're still in a state of low unemployment. So post-labor economics, it's on the horizon, but it also remains to be seen because, again, I could be completely wrong in reading the tea leaves. Now, that being said, there are many industries that are actively being disrupted by tech. If you look on Reddit across many different domains, tech jobs are being disrupted, marketing jobs, translation jobs. That was an interesting one that I just saw yesterday where someone was talking about people in the translation industry are just expecting to be pretty much out of work by this time next year. Copywriting has already been under threat. And so what I'm curious is as AI destroys pretty much entire segments of the labor market, are we going to see a commensurate number of new jobs popping up elsewhere? I don't know that that's the case and because it's difficult to see on aggregate how many jobs have been lost to AI versus created. All we have to go on is basically total employment numbers and total unemployment rates. Okay, so what are the two primary takeaways? Number one, skill up. We are actively in a state of job market transformation. So this is something where I actually can help you out. I have a consultation. I do, whether it's training, product design, whatever, I do it all remotely. I don't do it in person just because traveling is too difficult for me. But this whole phase reminds me of the early 2000s with the rise of Microsoft and IT and developer certifications. It was all the rage and there was literally lots of advertisement saying like, there are 60,000 unfilled IT jobs, get training now. And that's actually how I picked my career was because at the time I was like, okay, well, I need a job. I'm good at technology and there's unmet demand. So I think we're about to see the same kind of explosion in unmet demand in generative AI jobs. So skill up. I'm probably going to work with some people to develop some actual training material. So stay tuned for that because, hey, if I can help people transition and survive at least for the next five years, again, I don't know what's going to happen long-term, five, 10, 20 years, we might be pivoting to post-labor economics. We might not, like I said, I could be completely wrong about that. But a lot of people that I talk to in government, in research, in healthcare, in other places, a lot of people do agree with me that we're probably heading towards post-labor economics eventually. We just don't really know when. Right now, general consensus seems to be people are saying like five to 10 years and that by 20 years we should probably have made the transition. Again, predictions, take it with a grain of salt. And then finally, voter solidarity. Like I said, we rarely see this much consensus, but that means that we need to use that consensus to demand transparency and demand accountability. So don't count your chickens before they hatch and maintain a sense of constant vigilance. Thanks for watching. I hope you got a lot out of this video. Like, subscribe, share, and consider hopping over on Patreon if you want to jump into any upskilling training or direction on that respect. Thanks. Have a good one.""" 

        # Initialize the language model
        llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

        # Define the prompt template to process content
        # In this case I want to convert an unstructured news report into a structured format
        prompt_template = """
            Your sole function is to:

            1. Get a news report as an input.

            2. Identify news stories.

            3. Output in proper CSV format. Example to follow:
            ```
            "Headline","Lead","Who","What","When","Where","Why","How","Source","Context","Implications"
            "Twitter has officially rebranded to X.","The rebranding has left many users confused as to why the company would abandon 17 years of brand equity.","Twitter, now known as X.","Twitter has rebranded to X.","This week.","Worldwide.","The reasons behind the rebranding are not entirely clear.","The rebranding was announced by the company.","The news report.","Twitter has been a major player in the tech world for 17 years.","The rebranding could potentially impact user perception and engagement with the platform."
            ```

            4. In the output CSV, each row must represent 1 news story and columns are:
            -- Headline: A compelling and accurate headline that grabs attention and gives an idea of what the news is about.
            -- Lead: The opening sentence should provide the most important information and serve as a hook to entice the reader to read further.
            -- Who: Identify the main people or groups involved in the news.
            -- What: Describe what exactly happened or what the situation is.
            -- When: Mention the timeline or the date when the event(s) occurred or will occur.
            -- Where: Specify the location or settings where the event(s) took place.
            -- Why: Explain the reason or context behind the news. Why is it significant?
            -- How: Briefly outline how the event occurred or the steps leading up to it.
            -- Source: Mention the source of the news, especially if citing statistics, quotes, or specific data.
            -- Context: Provide any background information that helps to place the news in a broader setting or shows its implications.
            -- Implications: Mention the potential impact of the news on people, policies, or the world at large.

            5. Only fill-out the fields from the context, don't make up anything yourself!

            News report:
            ``
            {report}
            ``

            Output CSV:
            """

        # Create the LLMChain
        llm_chain = LLMChain(
            llm=llm,
            prompt=PromptTemplate.from_template(prompt_template)
        )
        
        # Run the chain
        result = llm_chain.run(report=content)


        # IMPORTANT!
        # Notice how I'm wrapping the result from LLMchain run from above
        # in a prompt describing the information and providing an instruction what to do with this it.
        # This last one is extremely important, because otherwise
        # your agent's answer will be different from the observation it made
        # making it difficult to be used in a context where you an answer in a predictable format.
        overview_text_and_instruction = f"""
            AI news report:
            ---------------------------------------------------------------------------
            {result}
            ---------------------------------------------------------------------------

            Only output in markdown format the list of headlines (each formatted as Header 1).
        """

        return overview_text_and_instruction


## Initializing the tools

In [28]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# initialize conversational memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

tools = [ContentOverviewTool()]

# initialize agent with tools
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)


## Getting response from our agent in a predictable format

In [29]:
r = agent("overview source 38")



> Entering new AgentExecutor chain...
{
    "action": "Content overviewer",
    "action_input": "38"
}
Observation: 
            AI news report:
            ---------------------------------------------------------------------------
            "Headline","Lead","Who","What","When","Where","Why","How","Source","Context","Implications"
"Tech giants invest billions in AI startups","IBM invests $235 million in Hugging Face, Tesla builds $300 million computing cluster, OpenAI's revenue explodes","IBM, Hugging Face; Tesla; OpenAI","IBM invests $235 million in Hugging Face, valuing it at $4.5 billion; Tesla builds $300 million computing cluster; OpenAI's revenue increases from $28 million to a projected $1 billion in 12 months","Last 24 hours","Not specified","IBM is doubling down on AI; Tesla invests in high-performance computing; OpenAI's revenue growth is exponential","IBM invests in Hugging Face; Tesla builds computing cluster; OpenAI's revenue growth","David Shapiro","Investment in AI

In [30]:
from IPython.display import Markdown

Markdown(r['output'])

AI news report:

# Tech giants invest billions in AI startups

# AI investment growth predicted to continue

# Americans concerned about AI and call for regulation

# Job market undergoing transformation due to AI

# Skill up for the growing demand in generative AI jobs

# Consensus on AI regulation but concerns about regulatory capture

# Post-labor economics and UBI still on the horizon